In [ ]:
import sys

sys.path.append("code")
import nbutils
import json

from vectorian.importers import StringImporter
from vectorian.embeddings import SentenceBertEmbedding
from vectorian.corpus import Corpus
from tqdm import tqdm

nlp = nbutils.make_nlp()

with open("../data/raw_data/gold.json", "r") as f:
    queries = json.loads(f.read())

bert_embedding = SentenceBertEmbedding(nlp)

def prepare_docs():
    if bert_embedding is not None:
        im = StringImporter(nlp, embeddings=[bert_embedding])
    else:
        im = StringImporter(nlp)

    docs = []

    for query in tqdm(queries, desc="Importing"):
        for m in query["matches"]:
            docs.append(im(
                m["context"],
                title=m["work"],
                author=m["author"],
                unique_id=m["id"]))
            
    Corpus(docs).save("../data/processed_data/corpus")
    
prepare_docs()